In [ ]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END

ImportError: cannot import name 'draw' from 'langgraph.pregel' (/opt/anaconda3/envs/ml/lib/python3.12/site-packages/langgraph/pregel/__init__.py)

In [21]:
class SimpleState(TypedDict):
    input_text: str
    output_text: str

### Custom Node

In [22]:
def process_input(state: SimpleState) -> SimpleState:
    output_text = state['input_text'].upper()
    return {'output_text':output_text}

In [23]:
state = {'input_text': 'hello world', 'output_text': ''}
process_input(state)

{'output_text': 'HELLO WORLD'}

In [24]:
def add_prefix(state: SimpleState):
    print("Current State[prefix]: ", state)
    output = "Hey, I have added something here. "+state['output_text']
    return {'output_text': output}

In [25]:
add_prefix(process_input(state))

Current State[prefix]:  {'output_text': 'HELLO WORLD'}


{'output_text': 'Hey, I have added something here. HELLO WORLD'}

In [26]:
def add_suffix(state: SimpleState):
    print("Current State[suffix]: ", state)
    output = state['output_text'] + ". I have added suffix ."
    return {'output_text': output}

In [27]:
add_suffix(add_prefix(process_input(state)))

Current State[prefix]:  {'output_text': 'HELLO WORLD'}
Current State[suffix]:  {'output_text': 'Hey, I have added something here. HELLO WORLD'}


{'output_text': 'Hey, I have added something here. HELLO WORLD. I have added suffix .'}

In [ ]:
def create_simple_graph():
    builder = StateGraph(SimpleState)

    # add the nodes
    builder.add_node("process_input", process_input)
    builder.add_node("add_prefix", add_prefix)
    builder.add_node("add_suffix", add_suffix)

    # connect the nodes
    builder.add_edge(START, "process_input")
    builder.add_edge("process_input","add_prefix")
    builder.add_edge("add_prefix","add_suffix")
    builder.add_edge("add_suffix", END)

    # makes it langchain runnable
    graph = builder.compile()

    return graph

In [29]:
graph = create_simple_graph()
graph

In [35]:
print(graph.get_graph().draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	process_input(process_input)
	add_prefix(add_prefix)
	add_suffix(add_suffix)
	__end__([<p>__end__</p>]):::last
	__start__ --> process_input;
	add_prefix --> add_suffix;
	add_suffix --> __end__;
	process_input --> add_prefix;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [37]:
# Initial state or user input
initial_state ={'input_text': 'hello'}
graph.invoke(initial_state)

Current State[prefix]:  {'input_text': 'hello', 'output_text': 'HELLO'}
Current State[suffix]:  {'input_text': 'hello', 'output_text': 'Hey, I have added something here. HELLO'}


{'input_text': 'hello',
 'output_text': 'Hey, I have added something here. HELLO. I have added suffix .'}